In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import gc
from sklearn.metrics import log_loss

In [36]:

train = pd.read_pickle(FILE.train_agg_v2.value)


test = pd.read_pickle(FILE.test_agg_v2.value)

print('train shape is: {}'.format(train.shape))    
print('test shape is: {}'.format(test.shape))




train shape is: (1001650, 633)
test shape is: (40024, 632)


# LGBM

In [3]:
from sklearn.model_selection import train_test_split
print_to_file = False 
test_run = False 

def get_time(timezone='America/New_York', time_format='%Y-%m-%d %H:%M:%S'):
    from datetime import datetime
    from dateutil import tz

    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz(timezone)

    utc = datetime.utcnow()

    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    est = utc.astimezone(to_zone)

    return est.strftime(time_format)

import sys, time
class Logger(object):
    def __init__(self, logtofile=True, logfilename='log'):
        self.terminal = sys.stdout
        self.logfile = "{}_{}.log".format(logfilename, int(time.time()))
        self.logtofile = logtofile

    def write(self, message):
        #         self.terminal.write(message)
        if self.logtofile:
            self.log = open(self.logfile, "a")
            self.log.write('[' + get_time() + '] ' + message)
            self.log.close()

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass


def divert_printout_to_file():
    sys.stdout = Logger(logfilename='logfile')

if print_to_file:
    divert_printout_to_file()  # note: comment this to use pdb

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, train_df, test_df, holdout, num_folds, submission_file_name, fe_img_name, stratified = False, debug= False, colsample=0.67, max_depth=8, num_leaves=31, min_child_samples=20, subsample=0.7, reg_lambda=0.3, lr=0.04, seed=1001, verbose=100, rounds=None, target='click'):
    print(train_df.shape, test_df.shape, holdout.shape)
    print('MEAN: train({}) vs holdout({}): '.format(len(train_df), len(holdout)), train_df[target].mean(), holdout[target].mean())
    # Divide in training/validation and test data
    if df is not None:
        train_df = df[df[target].notnull()]
        test_df = df[df[target].isnull()]
        print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
        del df
        gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
        
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    holdout_final_preds = np.zeros(holdout.shape[0])
    feature_importance_df = pd.DataFrame()
    feature_importance_gain_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in [target,'time','instance_id','index']]
    train_scores = []
    holdout_scores = []
    scores = []
    diff_val_holdout = []
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df[target])):
#         print('valid index : ',list(valid_idx)[:5])
        train_x, train_y = train_df[feats].iloc[train_idx], train_df[target].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df[target].iloc[valid_idx]
#         print('MEAN: train({}) vs valid({}): '.format(len(train_y), len(valid_y)), np.mean(train_y), np.mean(valid_y))

        clf = LGBMClassifier(
            nthread=10,
            n_estimators=30000,
            learning_rate=lr,
#             num_leaves=num_leaves,
#             colsample_bytree=colsample, # 0.67
#             subsample=subsample,
#             subsample_freq=0, ## disable subsampling
#             max_depth=max_depth,
#             reg_alpha=0.65,
#             reg_lambda=reg_lambda,
#             min_split_gain=0.0222415,
#             min_child_weight=39.3259775,
#             min_child_samples=min_child_samples,
            silent=-1,
            verbose=-1, )
        if rounds is not None:
            clf.n_estimators = rounds
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose)
            oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats])[:, 1] 
        else:
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose, early_stopping_rounds= 100)
            oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats], num_iteration=clf.best_iteration_)[:, 1] 
            
        holdout_final_preds += holdout_preds / folds.n_splits
        score = log_loss(valid_y, oof_preds[valid_idx])
        train_score = clf.best_score_['training']['binary_logloss']
        holdout_score = log_loss(holdout[target], holdout_preds)
        diff = abs(score - holdout_score)
        best_rounds = rounds if rounds is not None else clf.best_iteration_
        print('Fold %2d [%5d] AUC : ho: %.6f / te: %.6f / tr: %.6f (diff: %.6f)' % (n_fold + 1, best_rounds, holdout_score, score,  train_score, diff))
        scores.append(score)
        train_scores.append(train_score)
        holdout_scores.append(holdout_score)
        diff_val_holdout.append(diff)
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        fold_importance_gain_df = pd.DataFrame()
        fold_importance_gain_df["feature"] = feats
        fold_importance_gain_df["importance"] = clf.booster_.feature_importance(importance_type='gain')
        fold_importance_gain_df["fold"] = n_fold + 1
        feature_importance_gain_df = pd.concat([feature_importance_gain_df, fold_importance_gain_df], axis=0)
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    holdout_roc = log_loss(holdout[target], holdout_final_preds)
    holdout_mean = np.mean(holdout_scores)
    full_te_mean = np.mean(scores)
    full_tr_mean = np.mean(train_scores)
#     print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    print('Full HO score %.6f' % holdout_roc)
    print('FULL HO mean {:.6f}, std {:.6f}'.format(holdout_mean, np.std(holdout_scores)))
    print('FULL TE mean {:.6f}, std {:.6f}'.format(full_te_mean, np.std(scores)))
    print('FULL TR mean {:.6f}, std {:.6f}'.format(full_tr_mean, np.std(train_scores)))
    print('FULL DIFF mean {:.6f}, std {:.6f}'.format(np.mean(diff_val_holdout), np.std(diff_val_holdout)))
    # Write submission file and plot feature importance
    if not debug:
        print('saving...')
        test_df['predicted_score'] = sub_preds
        test_df[['instance_id', 'predicted_score']].to_csv(submission_file_name, index= False)
#     if not print_to_file:
#         display_importances(feature_importance_df, fe_img_name)
    feature_importance_df = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    feature_importance_gain_df = feature_importance_gain_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    return feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds

# Display/plot feature importance
def display_importances(feature_importance_df_, fe_img_name):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig(fe_img_name+'.png')


def convert_and_save_imp_df(fe_imp_df, dumpfilename):
    fe_imp_df_mean = fe_imp_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    pickle.dump(fe_imp_df_mean, open(dumpfilename,'wb'))
    
    
# seed = 6
# seed = 41
def select_holdout(train,test,shift=1,seed=41):
    samples_size = int(len(test) )
    assert shift in range(1,8)
    test_time_min = test.time.min()
    test_time_max = test.time.max()
    train_time_min = train.time.min()
    train_time_max = train.time.max()
    window_lower_bound = test_time_min - shift*60*60*24
    window_upper_bound = test_time_max - shift*60*60*24
    available_train = train[(train.time<=window_upper_bound) & (train.time>=window_lower_bound)]
    holdout = available_train.sample(n=samples_size,random_state=seed).copy()
    train_split_index = list(set(train.index) - set(holdout.index))
    train_split = train.iloc[train_split_index].copy()
    return train_split,holdout

In [ ]:
def runlgb(train, test, holdout):
    colsamples = [0.07]#[0.1,0.15,0.2]#[0.03,0.04,0.05,0.06,0.07,0.08]
    seeds = [20]#[300,4000,50000,600000,7000000,80000000,523445,31275479] # 20
    depth = [5]
    leaves = [16]
    min_child_sam = [20]#, 800]
    subsamples = [1]#0.8, 0.7, 0.6, 0.5, 0.4] # was 1
    reg_lambdas = [0.5]
    # lrs = lrs.tolist()
    lrs2 = [0.05]
    nfolds = 5
    rounds = [None] #[1000]#, 1300, 1600, 1900, 2200, 2500]
    for seed in seeds:
        for colsample in colsamples:
            for d in depth:
                for l in leaves:
                    for mcs in min_child_sam:
                        for subsample in subsamples:
                            for reg_lambda in reg_lambdas:
                                for lr in lrs2:
                                    for r in rounds:
                                        filename = 'fe_936_col{}_lr{}_n{}'.format(len(train.columns), lr, nfolds)
                                        print('#############################################')
                                        print(colsample, seed, d, l, mcs, subsample, reg_lambda, lr, 'nfolds:', nfolds)
                                        print('#############################################')
                                        numfeats = len(train.columns)
                                        with timer("Run LightGBM with kfold"):
                                            return kfold_lightgbm(None, train, test, holdout, nfolds, filename+'.csv', filename, colsample=colsample, verbose=100, max_depth=d, num_leaves=l, min_child_samples=mcs, subsample=subsample, reg_lambda=reg_lambda, lr=lr, seed=seed, stratified=True, rounds=r,debug=True)                                        

In [37]:
train_df,holdout = select_holdout(train,test,shift=1,seed=41)
print('train shape: {}, holdout shape: {}'.format(train_df.shape,holdout.shape))
print('train mean: {}, holdout mean: {}'.format(train.click.mean(),holdout.click.mean()))


feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train_df, test, holdout)

train shape: (961626, 633), holdout shape: (40024, 633)
train mean: 0.19845954175610242, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 633) (40024, 632) (40024, 633)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.41645	valid_1's binary_logloss: 0.417765
[200]	training's binary_logloss: 0.413944	valid_1's binary_logloss: 0.417083
[300]	training's binary_logloss: 0.412141	valid_1's binary_logloss: 0.416828
[400]	training's binary_logloss: 0.41057	valid_1's binary_logloss: 0.416762
[500]	training's binary_logloss: 0.409082	valid_1's binary_logloss: 0.416715
[600]	training's binary_logloss: 0.40769	valid_1's binary_logloss: 0.416672
[700]	training's binary_logloss: 0.406349	valid_1's binary_logloss: 0.41666
Early stopping, best iteration

# Select features

In [38]:
base_gain = feature_importance_gain_df.copy()
base_split = feature_importance_df.copy()

In [39]:
base_split.to_csv('../../data/features/agg_v2_select_feature_importance_split.csv',index=False)
base_gain.to_csv('../../data/features/agg_v2_select_feature_importance_gain.csv',index=False)

In [70]:
train = train_new.copy()
test = test_new.copy()

In [44]:
gain_fe = base_gain.iloc[:360]['feature'].values
split_fe = base_split.iloc[:360]['feature'].values
all_fe = set(gain_fe).intersection(set(split_fe))
print(len(all_fe))

327


In [45]:
all_fe = list(all_fe)
all_fe.append('instance_id')
all_fe.append('time')
train_fe = list(all_fe).copy()
train_fe.append('click')

train_new = train[train_fe].copy()
test_new = test[all_fe].copy()

In [35]:
train_df,holdout = select_holdout(train_new,test,shift=1,seed=41)
print('train shape: {}, holdout shape: {}'.format(train_df.shape,holdout.shape))
print('train mean: {}, holdout mean: {}'.format(train.click.mean(),holdout.click.mean()))
feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train_df, test_new, holdout)

train shape: (961626, 70), holdout shape: (40024, 70)
train mean: 0.19845954175610242, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 70) (40024, 69) (40024, 70)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.416424	valid_1's binary_logloss: 0.417741
[200]	training's binary_logloss: 0.413921	valid_1's binary_logloss: 0.417011
[300]	training's binary_logloss: 0.412181	valid_1's binary_logloss: 0.416886
[400]	training's binary_logloss: 0.410511	valid_1's binary_logloss: 0.416736
[500]	training's binary_logloss: 0.409065	valid_1's binary_logloss: 0.416724
[600]	training's binary_logloss: 0.407747	valid_1's binary_logloss: 0.416678
[700]	training's binary_logloss: 0.406459	valid_1's binary_logloss: 0.416652
[800]	training's binary_logloss

KeyboardInterrupt: 

In [36]:
train_df,holdout = select_holdout(train_new,test,shift=1,seed=41)
print('train shape: {}, holdout shape: {}'.format(train_df.shape,holdout.shape))
print('train mean: {}, holdout mean: {}'.format(train.click.mean(),holdout.click.mean()))
feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train_df, test_new, holdout)

train shape: (961626, 62), holdout shape: (40024, 62)
train mean: 0.19845954175610242, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 62) (40024, 61) (40024, 62)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.409218	valid_1's binary_logloss: 0.418057
[200]	training's binary_logloss: 0.402489	valid_1's binary_logloss: 0.418387
Early stopping, best iteration is:
[115]	training's binary_logloss: 0.40804	valid_1's binary_logloss: 0.418027
Fold  1 [  115] AUC : ho: 0.428232 / te: 0.418027 / tr: 0.408040 (diff: 0.010205)
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.409469	valid_1's binary_logloss: 0.416872
[200]	training's binary_logloss: 0.40302	valid_1's binary_logloss: 0.417178
Early s

In [39]:
train_df,holdout = select_holdout(train_new,test,shift=1,seed=41)
print('train shape: {}, holdout shape: {}'.format(train_df.shape,holdout.shape))
print('train mean: {}, holdout mean: {}'.format(train.click.mean(),holdout.click.mean()))
feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train_df, test_new, holdout)

train shape: (961626, 45), holdout shape: (40024, 45)
train mean: 0.19845954175610242, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 45) (40024, 44) (40024, 45)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.409279	valid_1's binary_logloss: 0.41808
[200]	training's binary_logloss: 0.402835	valid_1's binary_logloss: 0.418483
Early stopping, best iteration is:
[110]	training's binary_logloss: 0.408406	valid_1's binary_logloss: 0.418039
Fold  1 [  110] AUC : ho: 0.428234 / te: 0.418039 / tr: 0.408406 (diff: 0.010195)
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.409534	valid_1's binary_logloss: 0.416897
[200]	training's binary_logloss: 0.40328	valid_1's binary_logloss: 0.417231
Early s

In [46]:
train_new.to_pickle(FILE.train_agg_v2_select.value)
test_new.to_pickle(FILE.test_agg_v2_select.value)

In [ ]:
selected_col = ['make+creative_id_reverse_cumulative_count','nnt+orderid_count_in_next_n_time_unit_1hour','make+creative_id_reverse_cumulative_count','campaign_id+make_reverse_cumulative_count']

In [17]:
feature_importance_df

,feature,importance
0,model,1904.8
1,inner_slot_id,609.0
2,make,209.0
3,advert_name,153.2
4,osv,101.4
5,creative_id,81.2
6,creative_type,57.2
7,f_channel,51.0
8,advert_industry_inner,42.2
9,adid,40.0


In [7]:
feature_importance_df

,feature,importance
0,model,1934.0
1,inner_slot_id,612.8
2,make,218.2
3,advert_name,153.6
4,osv,101.0
5,creative_id,75.8
6,creative_type,57.2
7,f_channel,51.0
8,adid,48.0
9,advert_industry_inner,41.8
